In [1]:
import pandas as pd
import geopandas as gpd
import googlemaps
import requests
import json
from tqdm import tqdm
pd.set_option('display.max_columns', 500)

In [2]:
def load_api(path="confidential.json"):
	with open(path) as f:
		data = json.load(f)
	return data.get("api_key", "API key not found")
	
api_key = load_api()

In [3]:
# zip_tract = pd.read_excel("D:\Download\HVIdata\ZIP_TRACT_122024.xlsx")
# zip_tract = zip_tract[zip_tract['USPS_ZIP_PREF_STATE'] == 'NJ']
# zip_tract.to_csv("D:\Download\HVIdata\ZIP_TRACT_122024_NJ.csv", index=False)

In [4]:
zip_tract = pd.read_csv("D:\Download\HVIdata\data\ZIP_TRACT_122024_NJ.csv")
# zip_tract = zip_tract.sort_values(by=['TRACT', 'RES_RATIO'], ascending=[True, False])
# zip_tract = zip_tract.drop_duplicates(subset='TRACT', keep='first')
# zip_tract = zip_tract[['TRACT', 'ZIP']]
# zip_tract = zip_tract.rename(columns={'TRACT': 'GEOID', 'ZIP': 'ZCTA'})
zip_tract.to_csv("D:\Download\HVIdata\data\ZIP_TRACT_122024_NJ.csv", index=False)

In [5]:
zip_tract

,GEOID,ZCTA
0,34001000100,8401
1,34001000200,8401
2,34001000300,8401
3,34001000400,8401
4,34001000500,8401
...,...,...
2168,34041032101,8802
2169,34041032102,8886
2170,34041032200,8865
2171,34041032300,8804


In [47]:
nj_hvi = pd.read_csv('data/nj_hvi.csv')

In [49]:
nj_hvi = nj_hvi.merge(zip_tract, on=['GEOID', 'ZCTA'], how='inner')
nj_hvi.head()

,Adaptive_Capacity_Scale,Annual_PM25_Concentration,Asthma_Prevalence,CoronaryHeartDisease_Prevalence,COUNTYFP,Diabetes_Prevalence,Exposure_Scale,GEOID,HVI_Category,HVI_Scale,Impervious_to_Canopy_Ratio,NAME,NAMELSAD,O3_Exceedance_Days,ObjectID,Percent_Below_PovertyLine,Percent_Linguistic_Isolation,Percent_NoHealth+V1Insurance,Percent_Aged_Above_65,Percent_Aged_Below_5,Percent_with_Disability,Percent_Homes_Built_before_1960,Percent_in_Outdoor_Occupations,Percent_Living_Alone,Percent_NonWhite,Percent_Unemployed,Percent_without_HighSchoolDiploma,Sensitivity_Scale,STATEFP,Tract_Summ,TRACTCE,Shape__Area,Shape__Length,OBJECTID_1,ZCTA,Lat,Lon
0,1,2,3,1,31,1,1,34031256804,Low,1,1,2568.04,Census Tract 2568.04,2,1396,1,1,3,3,2,2,2,2,1,1,2,1,1,34,1,256804,"53,904,326.71","39,088.52",1,7480,41.098546,-74.393934
1,4,2,3,3,31,2,1,34031246300,Moderate,3,2,2463.00,Census Tract 2463,2,1836,4,3,4,2,3,2,3,4,3,3,4,3,3,34,1,246300,"18,543,764.50","32,287.12",2,7470,40.941149,-74.253146
2,5,2,5,3,31,5,3,34031181200,Moderate_High,4,5,1812.00,Census Tract 1812,2,1743,5,5,5,1,5,1,4,5,1,5,4,5,4,34,3,181200,"704,177.54","3,557.55",3,7514,40.923473,-74.144549
3,2,2,3,5,31,3,1,34031256803,Low,1,1,2568.03,Census Tract 2568.03,2,1402,2,2,3,5,4,1,2,3,5,2,1,2,3,34,1,256803,"26,408,937.79","30,301.58",4,7480,41.098546,-74.393934
4,1,2,2,3,31,2,1,34031246003,Low,1,2,2460.03,Census Tract 2460.03,2,1395,1,3,1,4,5,2,3,2,1,2,2,2,2,34,1,246003,"14,709,914.19","20,129.26",5,7470,40.941149,-74.253146


In [36]:
gmaps = googlemaps.Client(key=api_key)
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

def location_retrieve(zipcode, state):
    """Retrieve latitude and longitude for a given ZIP code with caching to reduce API calls."""    
    endpoint = f"{base_url}?components=country=us&address={zipcode},{state}&key={api_key}"
    response = requests.get(endpoint).json()
    if 'results' in response and response['results']:
        return response['results'][0]['geometry']['location']['lat'], response['results'][0]['geometry']['location']['lng']
    return None, None

In [51]:
nj_hvi["ZCTA"] = nj_hvi["ZCTA"].astype(str).str.zfill(5)

latitudes, longitudes = [], []
for zipcode in tqdm(nj_hvi["ZCTA"], desc="Fetching geolocation data"):
    lat, lon = location_retrieve(zipcode, "NJ")
    latitudes.append(lat)
    longitudes.append(lon)

# Assign results to DataFrame
nj_hvi["Lat"] = latitudes
nj_hvi["Lon"] = longitudes

Fetching geolocation data: 100%|██████████| 1818/1818 [03:32<00:00,  8.54it/s]


In [52]:
nj_hvi.head()


,Adaptive_Capacity_Scale,Annual_PM25_Concentration,Asthma_Prevalence,CoronaryHeartDisease_Prevalence,COUNTYFP,Diabetes_Prevalence,Exposure_Scale,GEOID,HVI_Category,HVI_Scale,Impervious_to_Canopy_Ratio,NAME,NAMELSAD,O3_Exceedance_Days,ObjectID,Percent_Below_PovertyLine,Percent_Linguistic_Isolation,Percent_NoHealth+V1Insurance,Percent_Aged_Above_65,Percent_Aged_Below_5,Percent_with_Disability,Percent_Homes_Built_before_1960,Percent_in_Outdoor_Occupations,Percent_Living_Alone,Percent_NonWhite,Percent_Unemployed,Percent_without_HighSchoolDiploma,Sensitivity_Scale,STATEFP,Tract_Summ,TRACTCE,Shape__Area,Shape__Length,OBJECTID_1,ZCTA,Lat,Lon
0,1,2,3,1,31,1,1,34031256804,Low,1,1,2568.04,Census Tract 2568.04,2,1396,1,1,3,3,2,2,2,2,1,1,2,1,1,34,1,256804,"53,904,326.71","39,088.52",1,07480,41.098546,-74.393934
1,4,2,3,3,31,2,1,34031246300,Moderate,3,2,2463.00,Census Tract 2463,2,1836,4,3,4,2,3,2,3,4,3,3,4,3,3,34,1,246300,"18,543,764.50","32,287.12",2,07470,40.941149,-74.253146
2,5,2,5,3,31,5,3,34031181200,Moderate_High,4,5,1812.00,Census Tract 1812,2,1743,5,5,5,1,5,1,4,5,1,5,4,5,4,34,3,181200,"704,177.54","3,557.55",3,07514,40.923473,-74.144549
3,2,2,3,5,31,3,1,34031256803,Low,1,1,2568.03,Census Tract 2568.03,2,1402,2,2,3,5,4,1,2,3,5,2,1,2,3,34,1,256803,"26,408,937.79","30,301.58",4,07480,41.098546,-74.393934
4,1,2,2,3,31,2,1,34031246003,Low,1,2,2460.03,Census Tract 2460.03,2,1395,1,3,1,4,5,2,3,2,1,2,2,2,2,34,1,246003,"14,709,914.19","20,129.26",5,07470,40.941149,-74.253146


In [12]:
print(nj_hvi.shape)
nj_hvi.columns

(1818, 38)


Index(['Adaptive_Capacity_Scale', 'Annual_PM25_Concentration',
       'Asthma_Prevalence', 'CoronaryHeartDisease_Prevalence', 'COUNTYFP',
       'Diabetes_Prevalence', 'Exposure_Scale', 'GEOID', 'HVI_Category',
       'HVI_Scale', 'Impervious_to_Canopy_Ratio', 'NAME', 'NAMELSAD',
       'O3_Exceedance_Days', 'ObjectID', 'Percent_Below_PovertyLine',
       'Percent_Linguistic_Isolation', 'Percent_NoHealth+V1Insurance',
       'Percent_Aged_Above_65', 'Percent_Aged_Below_5',
       'Percent_with_Disability', 'Percent_Homes_Built_before_1960',
       'Percent_in_Outdoor_Occupations', 'Percent_Living_Alone',
       'Percent_NonWhite', 'Percent_Unemployed',
       'Percent_without_HighSchoolDiploma', 'Sensitivity_Scale', 'STATEFP',
       'Tract_Summ', 'TRACTCE', 'Shape__Area', 'Shape__Length', 'OBJECTID_1',
       'ZCTA_x', 'Lat', 'Lon', 'ZCTA_y'],
      dtype='object')

In [ ]:
# hhi_data = pd.read_excel(rf"D:\Download\HVIdata\data\HHI Data 2024 United States.xlsx")
# hhi_data

In [44]:
# hhi_nj_fl_ca = hhi_data[hhi_data['STATE'].isin(['NJ', 'FL', 'CA'])]
# hhi_nj_fl_ca.to_csv(rf"D:\Download\HVIdata\data\hhi_nj_fl_ca.csv", index=False)
hhi_nj_fl_ca = pd.read_csv(rf"D:\Download\HVIdata\data\hhi_nj_fl_ca.csv")

In [ ]:
# hhi_nj_fl_ca["ZCTA"] = hhi_nj_fl_ca["ZCTA"].astype(str).str.zfill(5)

# # Apply function efficiently while reducing API calls
# latitudes, longitudes = [], []
# for zipcode in tqdm(hhi_nj_fl_ca["ZCTA"], desc="Fetching geolocation data"):
#     lat, lon = location_retrieve(zipcode, hhi_nj_fl_ca[hhi_nj_fl_ca["ZCTA"] == zipcode]["STATE"].iloc[0])
#     latitudes.append(lat)
#     longitudes.append(lon)

# # Assign results to DataFrame
# hhi_nj_fl_ca["Lat"] = latitudes
# hhi_nj_fl_ca["Lon"] = longitudes

Fetching geolocation data:   0%|          | 0/3298 [00:00<?, ?it/s]

Fetching geolocation data: 100%|██████████| 3298/3298 [09:13<00:00,  5.95it/s]


In [45]:
hhi_nj_fl_ca.columns

Index(['STATEFP10', 'STATE', 'STATE_ABV', 'ZCTA', 'GEOID10', 'MULTI_STATE',
       'POP', 'PR_HRI', 'F_HRI', 'LOW_EMS', 'P_NEHD', 'PR_NEHD', 'HHB_SCORE',
       'HHB_RANK', 'P_CHD', 'PR_CHD', 'F_CHD', 'P_OBS', 'PR_OBS', 'F_OBS',
       'P_DIABETES', 'PR_DIABETES', 'F_DIABETES', 'P_COPD', 'PR_COPD',
       'F_COPD', 'P_ASTHMA', 'PR_ASTHMA', 'F_ASTHMA', 'P_MNTHL', 'PR_MNTHL',
       'F_MNTHL', 'F_SEN_COUNT', 'SEN_SCORE', 'SEN_RANK', 'P_UNINSUR',
       'PR_UNINSUR', 'P_POV', 'PR_POV', 'P_UNEMP', 'PR_UNEMP', 'P_NOHSDP',
       'PR_NOHSDP', 'P_ISO', 'PR_ISO', 'P_ELP', 'PR_ELP', 'P_DISABL',
       'PR_DISABL', 'P_ODW', 'PR_ODW', 'P_AGE65', 'PR_AGE65', 'P_AGE5',
       'PR_AGE5', 'SOCIODEM_SCORE', 'SOCIODEM_RANK', 'P_IMPERV', 'PR_IMPERV',
       'P_TREEC', 'PR_TREEC', 'P_NOVEH', 'PR_NOVEH', 'P_MOBILE', 'PR_MOBILE',
       'P_RENT', 'PR_RENT', 'P_OZONE', 'PR_OZONE', 'P_PM25', 'PR_PM25',
       'NBE_SCORE', 'NBE_RANK', 'OVERALL_SCORE', 'OVERALL_RANK', 'Lat', 'Lon'],
      dtype='object')